In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS fleetops_project.silver;

CREATE TABLE IF NOT EXISTS fleetops_project.silver.driver_assignments_scd (
  assignment_sk    BIGINT GENERATED ALWAYS AS IDENTITY, 
  assignment_id    INT,                               
  driver_id        INT,
  truck_id         INT,
  is_active_source BOOLEAN, 
  effective_start  TIMESTAMP,                           
  effective_end    TIMESTAMP,                          
  is_current       BOOLEAN,
  row_hash         STRING                               
) USING DELTA;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW stage_assignments_raw AS
SELECT
  CAST(assignment_id AS INT) AS assignment_id,
  CAST(driver_id AS INT)     AS driver_id,
  CAST(truck_id AS INT)      AS truck_id,
  CAST(is_active AS BOOLEAN) AS is_active_source,
  COALESCE(CAST(updated_at AS TIMESTAMP), _ingested_at) AS src_ts
FROM fleetops_project.bronze.driver_assignments
WHERE assignment_id IS NOT NULL;

CREATE OR REPLACE TEMP VIEW stage_assignments_latest AS
SELECT * EXCEPT(rn)
FROM (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY assignment_id 
      ORDER BY src_ts DESC
    ) AS rn
  FROM stage_assignments_raw
)
WHERE rn = 1;

CREATE OR REPLACE TEMP VIEW stage_assignments_hash AS
SELECT
  *,
  sha2(concat_ws('||',
    CAST(driver_id AS STRING),
    CAST(truck_id AS STRING),
    CAST(is_active_source AS STRING)
  ), 256) AS source_hash
FROM stage_assignments_latest;

In [0]:
%sql
MERGE WITH SCHEMA EVOLUTION INTO fleetops_project.silver.driver_assignments_scd AS tgt
USING stage_assignments_hash AS src
ON  tgt.assignment_id = src.assignment_id
AND tgt.is_current    = TRUE  
WHEN MATCHED AND tgt.row_hash <> src.source_hash THEN
  UPDATE SET
    tgt.effective_end = src.src_ts,
    tgt.is_current    = FALSE;

INSERT INTO fleetops_project.silver.driver_assignments_scd (
  assignment_id, driver_id, truck_id, is_active_source,
  effective_start, effective_end, is_current, row_hash
)
SELECT
  s.assignment_id,
  s.driver_id,
  s.truck_id,
  s.is_active_source,
  s.src_ts                AS effective_start,
  TIMESTAMP('9999-12-31') AS effective_end,  
  TRUE                    AS is_current,
  s.source_hash           AS row_hash
FROM stage_assignments_hash s
LEFT JOIN fleetops_project.silver.driver_assignments_scd c
  ON c.assignment_id = s.assignment_id AND c.is_current = TRUE
WHERE c.assignment_id IS NULL       
   OR c.row_hash <> s.source_hash;     